In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mnist
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**EDA**

In [ ]:
X_train, X_test, y_train, y_test = mnist.train_images(), mnist.test_images(), mnist.train_labels(), mnist.test_labels()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
np.unique(y_train)

In [ ]:
X_train

In [ ]:
for i in range(9):
    plt.subplot(1, 9, i+1)
    # Display the image
    plt.imshow(X_train[i], cmap=plt.cm.jet) 
    
    # Set the title of the subplot to the corresponding label
    plt.title(f" y: {y_train[i]}")
     # Turn off axes for better presentation
    plt.axis('off')  
    
plt.show()

**Data Preprocessing**

In [ ]:
# Define a pixel normalization function
def pixel_normalization(image):
    return (image / 255)

In [ ]:
# Apply pixel normalization to the training and test sets
X_train = pixel_normalization(X_train)
X_test = pixel_normalization(X_test)

In [ ]:
# Expand the dimensions of the training and test data
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [ ]:
X_train.shape

**Model training**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)  # Change input channels from 3 to 1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
X_train = X_train.reshape(-1, 1, 28, 28)
y_train = y_train.reshape(-1, 1)
running_loss_list=[]


for epoch in range(2):  # Boucle sur le dataset plusieurs fois
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(zip(X_train, y_train)): # Utilisez la taille du lot pour déterminer le nombre d'itérations
       
        inputs = torch.from_numpy(inputs).unsqueeze(0).float()
        labels = torch.tensor(labels)
        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passe avant + rétropropagation + optimisation
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        
        # Afficher les statistiques
        running_loss += loss.item()
        if i % 2000 == 1999:  # Afficher toutes les 2000 itérations
            #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss_list.append(running_loss / 2000)
            running_loss = 0.0

print('Finished Training')
# Plot the loss values
plt.plot(running_loss_list, label='Training Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training Loss Over Iterations')
plt.legend()
plt.show()



In [ ]:
X_test = X_train.reshape(-1, 1, 28, 28)
y_test = y_train.reshape(-1, 1)

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in zip(X_test, y_test):
        inputs = torch.from_numpy(inputs).unsqueeze(0).float()
        labels = torch.tensor(labels)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy: %d %%' % accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Display the first 10 images from the test set
fig, axes = plt.subplots(1, 10, figsize=(20, 5))

for i in range(10):
    axes[i].imshow(X_test[i].reshape(28, 28), cmap='gray')  # Reshape the image to 28x28 and use grayscale colormap
    axes[i].set_title(str(y_test[i]))  # Set the title of the subplot to the corresponding label
    axes[i].axis('off')  # Turn off axes for better presentation

plt.show()

In [ ]:
import torch
# Assuming net is your trained model

# Assuming X_test contains the first 9 images and y_test contains their corresponding labels
X_test = X_train[:10].reshape(-1, 1, 28, 28)
y_test = y_train[:10].reshape(-1, 1)

with torch.no_grad():
    for i in range(10):
        inputs = torch.from_numpy(X_test[i]).unsqueeze(0).float()
        label = torch.tensor(y_test[i])
        output = net(inputs)
        _, predicted = torch.max(output.data, 1)
        print(f'Predicted class for image {i + 1}: {predicted.item()}, True class: {label.item()}')